In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel, AdditiveKernel, LinearKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

In [2]:
device = torch.device("cpu")

In [3]:
# --- 1. Загрузка данных ---
df = pd.read_csv("ourall.csv")
df = df[df['B4TOD4'] != 0.005]
df = df[df['OMEGA5'] != 15]
df = df[df['OMEGA5'] != -15]
PARAMS = [col for col in df.columns if col != "PT_LOSS"]
X = df[PARAMS].values
y = df["PT_LOSS"].values #.reshape(-1, 1)

# --- 2. min/max по параметрам ---
param_bounds = {col: (df[col].min(), df[col].max()) for col in PARAMS}

# --- 3. Нормализация данных (очень желательно для GPs) ---
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaler = StandardScaler()
# y_scaled = y_scaler.fit_transform(y).flatten()
# --- 4. Делим на train и pool ---
np.random.seed(42)
INIT_SIZE = 10
initial_idx = np.random.choice(len(X), size=INIT_SIZE, replace=False)
X_train, y_train = X_scaled[initial_idx], y[initial_idx]
X_pool, y_pool = (
    np.delete(X_scaled, initial_idx, axis=0),
    np.delete(y, initial_idx, axis=0),
)
print(X_pool,y_pool,len(X_pool))

[[0.  0.5 0.  ... 0.  0.  0. ]
 [0.  0.5 0.  ... 0.  0.  1. ]
 [0.  0.5 0.  ... 0.  0.2 0. ]
 ...
 [1.  0.  1.  ... 0.5 0.8 1. ]
 [1.  0.  1.  ... 0.5 1.  0. ]
 [1.  0.  1.  ... 0.5 1.  1. ]] [0.108975  0.0847613 0.061027  ... 0.0992762 0.116008  0.0890589] 3878


In [4]:
class GPRegressionModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = (
            ScaleKernel(MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])) +
            ScaleKernel(RBFKernel(ard_num_dims=train_x.shape[1]))
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [5]:
def make_kernel(idx, num_dims):
    if idx == 0:
        return ScaleKernel(RBFKernel(ard_num_dims=num_dims))
    elif idx == 1:
        return ScaleKernel(MaternKernel(nu=1.5, ard_num_dims=num_dims))
    elif idx == 2:
        return ScaleKernel(MaternKernel(nu=2.5, ard_num_dims=num_dims))
    else:
        raise ValueError("Committee kernel idx out of range")


# --- Обёртка для одной модели ---
class GPCommitteeModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# --- Обучаем комитет ---
def train_committee(X_train, y_train, num_models=3, num_iter=60, device="cpu"):
    committee = []
    for i in range(num_models):
        train_x = torch.tensor(X_train, dtype=torch.float32).to(device)
        train_y = torch.tensor(y_train, dtype=torch.float32).to(device)
        likelihood = GaussianLikelihood().to(device)
        kernel = make_kernel(i, X_train.shape[1])
        model = GPCommitteeModel(train_x, train_y, likelihood, kernel).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
        mll = ExactMarginalLogLikelihood(likelihood, model)
        model.train()
        likelihood.train()
        for it in range(num_iter):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            if torch.isnan(loss):
                print(f"NaN in model {i}, iter {it}, breaking")
                break
            loss.backward()
            optimizer.step()
        committee.append((model, likelihood))
    return committee

In [6]:
def qbc(committee, X_sample):
    # 1. Преобразуем в тензор с градиентами
    tx = torch.tensor(X_sample, dtype=torch.float32, requires_grad=True)

    preds = []
    for model, likelihood in committee:
        model.eval()
        likelihood.eval()
        x_query_t = tx.reshape(1, -1)
        mean = model(x_query_t).mean
        preds.append(mean)

    # 3. Вычисляем дисперсию предсказаний
    preds_tensor = torch.stack(preds)  # [n_models, n_samples]
    f_avg = preds_tensor.mean(dim=0)
    loss = torch.var(preds_tensor - f_avg, dim=0).mean()
    print(loss,'loss')
    # 4. Вычисляем градиенты
    tx.grad = None  # Очищаем предыдущие градиенты
    loss.backward()

    gradients = (
        tx.grad.detach().numpy() if tx.grad is not None else np.zeros_like(X_sample)
    )
    print("grads", gradients)
    return loss.item(), gradients


def NA_query_strategy(comittee, X_sample, X_pool):
    _, grads = qbc(comittee, X_sample)  # (N, M) -> N
    grads = [torch.tensor(row, dtype=torch.float32).unsqueeze(0) for row in grads]
    grads = torch.tensor(grads, dtype=torch.float32).numpy()
    print(grads,'grads')
    sign = np.sign(grads)
    print(sign,'sign')
    for ind in range(len(sign)):
        step = 0.01
        x_gen = X_sample[ind] + step * sign[ind]
        x_gen = np.clip(x_gen, 0.0, 1.0)
        dists = np.linalg.norm(X_pool - x_gen, axis=1)
        idx = np.argmin(dists)
    return idx, grads

In [13]:
import numpy as np
import torch


# --- Основной цикл ---
N_QUERIES = 50
num_models = 3  # число моделей в комитете
list_r2 = []
list_mae = []
list_elem = []
for it in range(N_QUERIES):
    # 1. Обучаем комитет (пример кода train_committee см. выше)
    committee = train_committee(X_train, y_train, num_models=num_models, num_iter=500)
    # 2. Выбираем точку по максимальному разногласию градиентов (NA-QBC)
    # for i in range(X_train.shape[0]):
    best_grad, best_ind = None, None
    for rnd_indx in range(X_train.shape[0]):
    # rnd_indx = np.random.choice(X_train.shape[0])
        idx, grad = NA_query_strategy(committee, X_train[rnd_indx], X_pool)
        if best_grad is None:
            best_grad = grad
            best_ind = idx
        # elif best_grad < sum(grad):
        #     best_grad = sum(grad)
        #     best_ind = idx
        how_1 = comparison = [abs(g1) > abs(g2) for g1, g2 in zip(grad, best_grad)]
        if len(how_1) > len(best_grad):
            best_grad = grad
            best_ind = idx
    idx = best_ind
    X_train = np.vstack([X_train, X_pool[idx]])
    y_train = np.append(y_train, y_pool[idx])
    X_pool = np.delete(X_pool, idx, axis=0)
    y_pool = np.delete(y_pool, idx, axis=0)
    print(f"Step {it + 1}: added point {idx}, pool left: {len(X_pool)}")
    preds_list = []
    # for model, likelihood in committee:
    #     model.eval()
    #     likelihood.eval()
    #     X_hold = torch.tensor(X_pool, dtype=torch.float32).to(device)
    #     with torch.no_grad(), gpytorch.settings.fast_pred_var():
    #         preds = model(X_hold)
    #         y_pred = preds.mean.cpu().numpy().flatten()
    #         y_true = y_pool
    #     preds_list.append(y_pred)
    # y_pred = np.mean(preds_list, axis=0)
    # mae = mean_absolute_error(y_true, y_pred)
    # r2 = r2_score(y_true, y_pred)
    # list_r2.append(r2)
    # list_mae.append(mae)
    # list_elem.append(it)
    for model, likelihood in committee:
        model.eval()
        likelihood.eval()
        X_hold = torch.tensor(X_train, dtype=torch.float32).to(device)
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            preds = model(X_hold)
            y_pred = preds.mean.cpu().numpy().flatten()
            y_true = y_train
        preds_list.append(y_pred)
    y_pred = np.mean(preds_list, axis=0)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    list_r2.append(r2)
    list_mae.append(mae)
    list_elem.append(it)
    print(f"Final MAE: {mae:.4f}, R2: {r2:.4f}")
    # print(f"Final MAE: {mae:.4f}, R2: {r2:.4f}")


tensor(1.7004e-07, grad_fn=<MeanBackward0>) loss
grads [-1.2247035e-06 -1.1446273e-05 -2.2081349e-05  2.9464923e-06
 -5.0549438e-06  1.5494745e-05  4.5846547e-05]
[-1.2247035e-06 -1.1446273e-05 -2.2081349e-05  2.9464923e-06
 -5.0549438e-06  1.5494745e-05  4.5846547e-05] grads
[-1. -1. -1.  1. -1.  1.  1.] sign
tensor(3.2476e-06, grad_fn=<MeanBackward0>) loss
grads [ 3.3342192e-06 -8.9873625e-05  6.0001394e-04 -1.7901872e-05
  6.7758934e-05 -1.2131255e-04 -2.3981062e-05]
[ 3.3342192e-06 -8.9873625e-05  6.0001394e-04 -1.7901872e-05
  6.7758934e-05 -1.2131255e-04 -2.3981062e-05] grads
[ 1. -1.  1. -1.  1. -1. -1.] sign
tensor(6.1287e-06, grad_fn=<MeanBackward0>) loss
grads [-2.5645788e-05  2.7449342e-04 -3.5056350e-05  1.1529815e-05
 -1.2369631e-04  1.8526912e-04  2.4901610e-04]
[-2.5645788e-05  2.7449342e-04 -3.5056350e-05  1.1529815e-05
 -1.2369631e-04  1.8526912e-04  2.4901610e-04] grads
[-1.  1. -1.  1. -1.  1.  1.] sign
tensor(1.8983e-06, grad_fn=<MeanBackward0>) loss
grads [-9.22170

In [15]:
print(list_elem)
print(list_mae)
print(list_r2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[0.0055074114761395855, 0.0054175408584038395, 0.004915205482846038, 0.0031498422794989166, 0.00418007968781471, 0.007665706616324183, 0.003208345987928612, 0.003894952662512988, 0.003041399317708766, 0.0037454991622257227, 0.0025403137568791694, 0.002738050042178413, 0.0026164628631653976, 0.002904046390563249, 0.002955447218437195, 0.0026986694263384887, 0.003660490941360261, 0.003928791634784426, 0.0030874707118346777, 0.0029798259938510264, 0.0029334154312410663, 0.0030120509356573225, 0.0031876609569881902, 0.003302307887317153, 0.003255483116205761, 0.0031281641614980173, 0.0031223783356447484, 0.003084270511471598, 0.0031560971623335135, 0.0031528913915944102, 0.0032380971984281776, 0.0030239813856261116, 0.0030459747215415157, 0.003063177628586509, 0.0031415381891822815, 0.00291612990223221